In [6]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import re
import pandas_gbq

In [2]:
client=bigquery.Client()

In [3]:
query = "select * from `basedosdados.br_mp_pep.cargos_funcoes`ORDER BY ano"

In [148]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Dados Brutos\\ENAP - ações afirmativas')


In [ ]:
df = pd.read_csv('corte_carreiras.csv', decimal=',', sep=';')
df.info()


In [150]:
df['Carreiras'] = df['Carreiras'].str.replace('\n', ' ') #Primeiramente, tratamento a coluna 'Carreiras' 

In [151]:
df['Carreiras'] = df['Carreiras'].str.title() 

In [152]:
df=df.dropna(how='all')

In [153]:
df.sort_values(by=['Total'], ascending=False).head(11)

,Carreiras,Ampla Concorrência,Cotista,Total,% Cotista
0,Assistente Em Administracao,5019.0,1271.0,6290.0,0.20
1,Professor Ens Basico Tecn Tecn,4357.0,571.0,4928.0,0.12
2,Tecnico De Laboratorio Area,1411.0,163.0,1574.0,0.10
3,Policial Rodoviario Federal,763.0,176.0,939.0,0.19
4,Aux Em Administracao,707.0,174.0,881.0,0.20
5,Tec De Tecnologia Da Informaca,702.0,96.0,798.0,0.12
6,Agente De Policia Federal 3A C,588.0,100.0,688.0,0.15
7,Tec Infor Geograficas E Estati,534.0,71.0,605.0,0.12
8,Administrador,493.0,88.0,581.0,0.15
9,Tecnico Em Assuntos Educaciona,385.0,98.0,483.0,0.20


In [155]:
carreiras_selecionadas = df.sort_values(by=['Total'], ascending=False).head(11)['Carreiras']

In [156]:
df['carreiras_selecionadas'] =  np.where(df['Carreiras'].isin(carreiras_selecionadas) ,df['Carreiras'],'Outros')

In [27]:
df.columns

Index(['Carreiras', 'Ampla Concorrência', 'Cotista', 'Total', '% Cotista',
       'carreiras_selecionadas'],
      dtype='object')

In [157]:
df1= df[['carreiras_selecionadas', 'Ampla Concorrência', 'Cotista','Total','Carreiras']].melt(['carreiras_selecionadas','Total','Carreiras'], var_name= 'tipo_ingresso', value_name = 'valor')

In [165]:
#### agrupando por selecionados e outros
x=df1.pivot_table(index=['carreiras_selecionadas','tipo_ingresso', 'Carreiras','Total'], values='valor',aggfunc=np.sum)
y = pd.DataFrame(x)
df = y.reset_index() ##inserir a contagem como coluna
#y = y.rename(columns={0:'count'})

In [170]:
#### agrupando por selecionados e outros
x=df1[df1['carreiras_selecionadas']=='Outros'].pivot_table(index=['carreiras_selecionadas','tipo_ingresso'], values='valor',aggfunc=np.sum)
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
#y = y.rename(columns={0:'count'})
y

,carreiras_selecionadas,tipo_ingresso,valor
0,Outros,Ampla Concorrência,9376.0
1,Outros,Cotista,1523.0


In [ ]:
df['prop'] = df['valor']/df['Total']

In [185]:
df['prop'] = np.where((df['carreiras_selecionadas']=="Outros") & (df['tipo_ingresso']=='Ampla Concorrência') ,0.86026,df['prop'])
df['prop'] = np.where((df['carreiras_selecionadas']=="Outros") & (df['tipo_ingresso']=="Cotista") ,0.13973,df['prop'])

In [192]:
df[df['carreiras_selecionadas']!='Outros']

,carreiras_selecionadas,tipo_ingresso,Carreiras,Total,valor,prop
0,Administrador,Ampla Concorrência,Administrador,581.0,493.0,0.848537
1,Administrador,Cotista,Administrador,581.0,88.0,0.151463
2,Agente De Policia Federal 3A C,Ampla Concorrência,Agente De Policia Federal 3A C,688.0,588.0,0.854651
3,Agente De Policia Federal 3A C,Cotista,Agente De Policia Federal 3A C,688.0,100.0,0.145349
4,Assistente Em Administracao,Ampla Concorrência,Assistente Em Administracao,6290.0,5019.0,0.797933
5,Assistente Em Administracao,Cotista,Assistente Em Administracao,6290.0,1271.0,0.202067
6,Aux Em Administracao,Ampla Concorrência,Aux Em Administracao,881.0,707.0,0.802497
7,Aux Em Administracao,Cotista,Aux Em Administracao,881.0,174.0,0.197503
442,Policial Rodoviario Federal,Ampla Concorrência,Policial Rodoviario Federal,939.0,763.0,0.812567
443,Policial Rodoviario Federal,Cotista,Policial Rodoviario Federal,939.0,176.0,0.187433


In [173]:
1523.0/10899.0

0.13973759060464264

In [183]:
9376.0/10899.0

0.8602624093953574

In [167]:
df['tipo_ingresso'].unique()

array(['Ampla Concorrência', 'Cotista'], dtype=object)

In [168]:
df

,carreiras_selecionadas,tipo_ingresso,Carreiras,Total,valor,prop
0,Administrador,Ampla Concorrência,Administrador,581.0,493.0,0.848537
1,Administrador,Cotista,Administrador,581.0,88.0,0.151463
2,Agente De Policia Federal 3A C,Ampla Concorrência,Agente De Policia Federal 3A C,688.0,588.0,0.854651
3,Agente De Policia Federal 3A C,Cotista,Agente De Policia Federal 3A C,688.0,100.0,0.145349
4,Assistente Em Administracao,Ampla Concorrência,Assistente Em Administracao,6290.0,5019.0,0.797933
...,...,...,...,...,...,...
451,Tecnico De Laboratorio Area,Cotista,Tecnico De Laboratorio Area,1574.0,163.0,0.103558
452,Tecnico Do Seguro Social,Ampla Concorrência,Tecnico Do Seguro Social,383.0,226.0,0.590078
453,Tecnico Do Seguro Social,Cotista,Tecnico Do Seguro Social,383.0,157.0,0.409922
454,Tecnico Em Assuntos Educaciona,Ampla Concorrência,Tecnico Em Assuntos Educaciona,483.0,385.0,0.797101


# Rascunho 

In [46]:
def transformar(nome):
    nome = re.sub(r"\bTecnico\b", "Técnico", nome)
    nome = re.sub(r"\bEmadministracao\b", "Em Administração", nome)
    nome = re.sub(r"\bProfessor Ens Basico Tecntecn\b", "Professor Ensino Básico Técnico Téc.", nome)
    nome = re.sub(r"\bLaboratorioarea\b", "Laboratório-Área", nome)
    nome = re.sub(r"\bPolicial Rodoviariofederal\b", "Policial Rodoviário Federal", nome)
    nome = re.sub(r"\bAux Em Administracao\b", "Auxiliar Em Administração", nome)
    nome = re.sub(r"\bTec De Tecnologia Dainformaca\b", "Técnico De Tecnologia Da Informação", nome)
    nome = re.sub(r"\bAgente De Policia Federal3A C\b", "Agente De Polícia Federal 3A C", nome) 
    nome = re.sub(r"\bTec Infor Geograficas Eestati\b", "Técnico em Informações Geográficas E Estatíticas", nome)
    nome = re.sub(r"\bAssuntoseducaciona\b", "Assuntos Educacionais", nome)
    nome = re.sub(r"\bMedico-Arfa\b", "Médico-Área", nome)
    nome = re.sub(r"\bEngenheiro-Area\b", "Engenheiro-Área", nome)
    nome = re.sub(r"\bBibliotecario-Documentalista\b", "Bibliotecário Documentalista", nome)
    nome = re.sub(r"\bAnalista De Tec Dainformacao\b", "Analista De Tecnologia Da Informação", nome)
    nome = re.sub(r"\bAuditor Fiscal Federalagropec\b", "Auditor Fiscal Federal Agropecuário", nome)
    nome = re.sub(r"\bTradutor Interprete Delinguag\b", "Tradutor Intérprete De Língua", nome)
    nome = re.sub(r"\bEnfermagem\b", "Enfermagem", nome)
    nome = re.sub(r"\bEnfermeiro-Area\b", "Enfermeiro-Área", nome)
    nome = re.sub(r"\bTecnologo-Formacao\b", "Tecnólogo-Formação", nome)
    nome = re.sub(r"\bPedagogo-Area\b", "Pedagogo-Área", nome)
    nome = re.sub(r"\bPsicologo-Area\b", "Psicólogo-Área", nome)
    nome = re.sub(r"\bUniao\b", "União", nome)
    nome = re.sub(r"\bLaboratorio\b", "Laboratório", nome)
    nome = re.sub(r"\bSecretario\b", "Secretário", nome)
    nome = re.sub(r"\bAgropecuaria\b", "Agropecuária", nome)
    nome = re.sub(r"\bDelegado De Policiafederal 3A\b", "Delegado De Polícia Federal 3A", nome)
    nome = re.sub(r"\bAnalista Plan Gest Infinfor G\b", "Analista Planejamento Gestão Informação G.", nome)
    nome = re.sub(r"\bPesq Em Propriedadeindustrial\b", "Pesquisador Em Propriedade Industrial", nome)
    nome = re.sub(r"\bNutricionista-Habilitacao\b", "Nutricionista Habilitação", nome)
    nome = re.sub(r"\bEletrotecnica\b", "Eletrotécnica", nome)
    nome = re.sub(r"Analista Em Ciencia Etecnolog", "Analista Em Ciência E Tecnologia", nome)
    nome = re.sub(r"\bTec Em Seguranca Dotrabalho\b", "Técnico Em Segurança Do Trabalho", nome)
    nome = re.sub(r"\bEscrivao De Policiafederal 3A\b", "Escrivão De Polícia Federal 3A", nome)
    nome = re.sub(r"\bAnalista De Planejamentoe Orc\b", "Analista De Planejamento E Orçamento", nome)
    nome = re.sub(r"\bMedico Veterinario\b", "Médico Veterinário", nome)
    nome = re.sub(r"\bTecnol Inf Geograficaestatist\b", "Tecnólogo Em Informação Geográfica Estatística", nome)
    nome = re.sub(r"\bMecanica\b", "Mecânica", nome)
    nome = re.sub(r"\bContramestre-Oficio\b", "Contramestre-Ofício", nome)
    nome = re.sub(r"\bTec E Reg De Serv Detransa Aq\b", "Técnico E Regulador De Serviços Detransação Aquaviária", nome)
    nome = re.sub(r"\bDefensor Publico Da Uniao\b", "Defensor Público Da União", nome)
    nome = re.sub(r"\bPerito Criminal Federal 3Acla\b", "Perito Criminal Federal 3A CLA", nome)
    nome = re.sub(r"\bPesquisador Em Saudepublica\b", "Pesquisador Em Saúde Pública", nome)
    nome = re.sub(r"\bAnalista Tecnicoadministrativ\b", "Analista Técnico Administrativo", nome)
    nome = re.sub(r"\bOdontologo - 40 Horas\b", "Odontólogo - 40 Horas", nome)
    nome = re.sub(r"\bTecnologista Em Propriedindus\b", "Tecnologista Em Propriedade Industrial", nome)
    nome = re.sub(r"\bEsp Em Reg De Serv Pub Detele\b", "Esp. Em Reg. De Serv. Púb. Tele.", nome) 
    nome = re.sub(r"\bGraficas\b", "Gráficas", nome)
    nome = re.sub(r"\bAssistente Em Ciencia Etecnol\b", "Assistente Em Ciência E Tecnologia", nome)
    nome = re.sub(r"\bEsp Em Reg De Serv Detransp A\b", "Esp. Em Reg. De Serv. De Transp. A", nome) 
    nome = re.sub(r"\bEdificacoes\b", "Edificações", nome)
    nome = re.sub(r"\bAgronomo\b", "Engenheiro Agrônomo", nome)
    nome = re.sub(r"\bSaude Publica\b", "Saúde Pública", nome)
    nome = re.sub(r"\bAgente De Insp Sanit Indprod\b", "Agente De Insp. Sanit. Ind. Prod", nome)
    nome = re.sub(r"\bFarmaceutico\b", "Farmacêutico", nome)
    nome = re.sub(r"\bBiologo\b", "Biólogo", nome)
    nome = re.sub(r"\bEngenheiro De Seg Dotrabalho\b", "Engenheiro De Seg. Do Trabalho", nome)
    nome = re.sub(r"\bDeedificios\b", "De Edifícios", nome)
    nome = re.sub(r"Programador Visual", "Programador Visual", nome)
    nome = re.sub(r"\bRelacoes Publicas\b", "Relações Públicas", nome)
    nome = re.sub(r"\bAnalista Exec Emmetrologia E\b", "Analista Exec. Em Metrologia E.", nome)
    nome = re.sub(r"\bAuxiliar Em Assuntoseducacion\b", "Auxiliar Em Assuntos Educacionais", nome)
    nome = re.sub(r"\bTec Em Regulacao Deaviacao Ci\b", "Técnico Em Regulação De Aviação C.", nome)
    nome = re.sub(r"\bMestre De Edif Einfraestrutur\b", "Mestre De Edif. E Infraestrutura", nome)
    nome = re.sub(r"\bDepoliticas\b", "Analista Técnico De Políticas", nome)
    nome = re.sub(r"\bElaticin\b", "E Latícínios", nome)
    nome = re.sub(r"\bQuimica\b", "Química", nome)
    nome = re.sub(r"\bFarmaceutico-Habilitacao\b", "Farmacêutico-Habilitação", nome)
    nome = re.sub(r"\bEletronica\b", "Eletrônica", nome)
    nome = re.sub(r"\bAssist De Tecnologia Dainform\b", "Assist. De Tecnologia Da Informação", nome)
    nome = re.sub(r"\bQuimico\b", "Químico", nome)
    nome = re.sub(r"\bSaude\b", " Saúde", nome)
    nome = re.sub(r"\bTec Em Eletroeletronica\b", "Técnico Em Eletroeletrônica", nome)
    nome = re.sub(r"\bOperador De Maqagricolas\b", "Operador De Máquinas Agrícolas", nome)
    nome = re.sub(r"\bPapiloscopista Polfederal 3A\b", "Papiloscopista Pol. Federal 3A", nome)
    nome = re.sub(r"\bMusico\b", "Músico", nome)
    nome = re.sub(r"\bFarmacia\b", "Técnico Em Farmácia", nome)
    nome = re.sub(r"\bAssist Tecnico De Gestaoem Sa\b", "Assistente Técnico De Gestão Em Saúde", nome)
    nome = re.sub(r"\bTec Em Regul De Saudesuplemen\b", "Técnico Em Regul. De Saúde Suplementar", nome)
    nome = re.sub(r"\bEsp Em Regulacao Deaviacao Ci\b", "Esp. Em Regulação De Aviação C.", nome)
    nome = re.sub(r"\bMuseologo\b", "Museólogo", nome)
    nome = re.sub(r"\bEstatistico\b", "Estatístico", nome)
    nome = re.sub(r"\bEmeletromecanica\b", "Técnico Em Eletromecânica", nome)
    nome = re.sub(r"\bTec Em Anatomia Enecropsia\b", "Técnico Em Anatomia E Necropsia", nome)
    nome = re.sub(r"\bTec Em Telecomunicacao\b", "Técnico Em Telecomunicação", nome)
    nome = re.sub(r"\bAssistente Exec Em Metrole Qu\b", "Assistente Exec. Em Metrol. e Q.", nome)
    nome = re.sub(r"\bFarmaceutico Bioquimico\b", "Farmacêutico Bioquímico", nome)
    nome = re.sub(r"\bOperador De Est Detratam Agua\b", "Operador De Est. De Tratam. Água", nome)
    nome = re.sub(r"\bTradutor Interprete\b", "Tradutor Intérprete", nome)
    nome = re.sub(r"\bTec Reg Pet E Deriv Alccomb\b", "Técnico Reg. Pet. E Deriv. Alc. Comb.", nome)
    nome = re.sub(r"Fisioterapeuta\b", "Fisioterapeuta", nome)
    nome = re.sub(r"\bOperador De Camera Decinema E\b", "Operador De Câmera De Cinema E", nome)
    nome = re.sub(r"\bAnalista Em Tecnologiada Info\b", "Analista Em Tecnologia Da Info", nome)
    nome = re.sub(r"\bAgente De Policia Federal2A C\b", "Agente De Polícia Federal 2A Classe", nome)
    nome = re.sub(r"\bTecnicoespecializa\b", "Técnico Especializado", nome)
    nome = re.sub(r"\bPesquisador-Tecnologistametr\b", "Pesquisador-Tecnologista Met.", nome)
    nome = re.sub(r"\bGeologo\b", "Geólogo", nome)
    nome = re.sub(r"\bAuxiliar De Creche\b", "Auxiliar De Creche", nome)
    nome = re.sub(r"\bMineracao\b", "Mineração", nome)
    nome = re.sub(r"\bTec Reg De Serv Pub Detelecom\b", "Técnico Reg. De Serv. Púb. De Telecom.", nome)
    nome = re.sub(r"\bAux De Veterinaria Ezootecnia\b", "Auxiliar De Veterinária E Zootecnia", nome)
    nome = re.sub(r"\bInstrumentadorcirurgico\b", "Instrumentador Cirúrgico", nome)
    nome = re.sub(r"\bProgramador De Radio Etelevis\b", "Programador De Rádio E Televisão", nome)
    nome = re.sub(r"\bFonoaudiologo\b", "Fonoaudiólogo", nome)
    nome = re.sub(r"\bTec Equip Medicoodontologico\b", "Técnico Equip. Médico-Odontológico", nome)
    nome = re.sub(r"\bEscrivao De Policiafederal 2A\b", "Escrivão De Polícia Federal 2A", nome)
    nome = re.sub(r"\bTec Em Nutricao Edietetica\b", "Técnico Em Nutrição E Dietética", nome)
    nome = re.sub(r"\bGeografo\b", "Geógrafo", nome)
    nome = re.sub(r"\bRefrigeracao\b", "Refrigeração", nome)
    nome = re.sub(r"\bFisico\b", "Físico", nome)
    nome = re.sub(r"\bNivel Superior", "Nível Superior", nome)
    nome = re.sub(r"\bComunicacaosocial\b", "Comunicação Social", nome)
    nome = re.sub(r"\bAgente De Pol Federalclasse E\b", "Agente De Polícia Federal Classe E", nome)
    nome = re.sub(r"\bPublicitario\b", "Publicitário", nome)
    nome = re.sub(r"\bEducacaofisica\b", "Educação Física", nome)
    nome = re.sub(r"\bAgropecuaria\b", "Auxiliar De Agropecuária", nome)
    nome = re.sub(r"\bProducao\b", "Produção", nome)
    nome = re.sub(r"Escrivao De Pol Federalclasse", "Escrivão De Polícia Federal Classe", nome)
    nome = re.sub(r"\bDesenhista De Artesgraficas\b", "Desenhista De Artes Gráficas", nome)
    nome = re.sub(r"\bEditor De Publicacoes\b", "Editor De Publicações", nome)
    nome = re.sub(r"\bFotografo\b", "Fotógrafo", nome)
    nome = re.sub(r"\bOperador De Radiotelecomunica\b", "Operador De Radiotelecomunicações", nome)
    nome = re.sub(r"\bLaboratorio\b", "Laboratório", nome)
    nome = re.sub(r"\bEconomiadomestica\b", "Economia Doméstica", nome)
    nome = re.sub(r"\bAssistente De Direcao Eproduc\b", "Assistente De Direção E Produção", nome)
    nome = re.sub(r"\bBiomedico\b", "Biomédico", nome)
    nome = re.sub(r"\bDiretor De Artes Cenicas\b", "Diretor De Artes Cênicas", nome)
    nome = re.sub(r"\bCoreografo\b", "Coreógrafo", nome)
    nome = re.sub(r"\bEngenheiro\b", "Engenheiro", nome)
    nome = re.sub(r"\bOtica\b", "Técnico Em Ótica", nome)
    nome = re.sub(r"\bAfinador De Inst Musicais\b", "Afinador De Instrumentos Musicais", nome)
    nome = re.sub(r"\bAgente De Telecomuni Eeletric\b", "Agente De Telecomunicação E Elétrica", nome)
    nome = re.sub(r"\bAnalista Tributario Recfedera\b", "Analista Tributário RecFedera", nome)
    nome = re.sub(r"\bArqueologo\b", "Arqueólogo", nome)
    nome = re.sub(r"\bAssistente Tecnicoadministrat\b", "Assistente Técnico Administrativo", nome)
    nome = re.sub(r"\bHialotecnico\b", "Hialotécnico", nome)
    nome = re.sub(r"\bMatematico\b", "Matemático", nome)
    nome = re.sub(r"\bMedico\b", "Médico", nome)
    nome = re.sub(r"\bMusico-Terapeuta\b", "Músico-Terapeuta", nome)
    nome = re.sub(r"\bTec Em Instrumentacao\b", "Técnico Em Instrumentação", nome)
    nome = re.sub(r"\bPapiloscopista Pol Fedclasse\b", "Papiloscopista Pol Fed Classe", nome)
    nome = re.sub(r"\b24Horas\b", "24 Horas", nome)
    nome = re.sub(r"\bTecnologo Emcooperativismo\b", "Tecnólogo Em Cooperativismo", nome)
    nome = re.sub(r"\bAgente De Policia Federalla C\b", "Agente De Polícia Federal Classe C", nome)
    nome = re.sub(r"\bVigilancia\b", "Vigilância", nome)
    nome = re.sub(r"\bAgente Federal Deexecucao Pen\b", "Agente Federal De Execução Penal", nome)
    nome = re.sub(r"\bAnalista De Infra-Estrutura\b", "Analista De Infraestrutura", nome)
    nome = re.sub(r"\bAntropologo\b", "Antropólogo", nome)
    nome = re.sub(r"\bAssistente Tecnico-Administrat\b", "Assistente Técnico-Administrativo", nome)
    nome = re.sub(r"\bBibliotecario\b", "Bibliotecário", nome)
    nome = re.sub(r"\bCenotecnico\b", "Cenotécnico", nome)
    nome = re.sub(r"\bEspecialista Emmetrologia E Q\b", "Especialista Em Metrologia E Q.", nome)
    nome = re.sub(r"\bMecanico De Montagem Emanuten\b", "Mecânico De Montagem E Manutenção", nome)
    nome = re.sub(r"\bOceanologo\b", "Oceanólogo", nome)
    nome = re.sub(r"\bOperador De Centralhidreletri\b", "Operador De Central Hidrelétrica", nome)
    nome = re.sub(r"\bPapiloscopista Polfederal 2A\b", "Papiloscopista Pol Federal 2A", nome)
    nome = re.sub(r"\bPsicologo\b", "Psicólogo", nome)
    nome = re.sub(r"\bTec Em Informacoeseducacionai\b", "Técnico Em Informações Educacionais", nome)
    nome = re.sub(r"\bTec Em Manutencao Deaudio Vid\b", "Técnico Em Manutenção De Áudio e Vídeo", nome)
    nome = re.sub(r"\bTec Em Propriedadeindustrial\b", "Técnico Em Propriedade Industrial", nome)
    nome = re.sub(r"\bTec Em Reabilit Oufisioterapi\b", "Técnico Em Reabilitação Fisioterapêutica", nome)
    nome = re.sub(r"\bTec Especializado Ii\b", "Técnico Especializado II", nome)
    nome = re.sub(r"\bTec Suporte Infra-Estrutde Tr\b", "Técnico de Suporte Infraestrutura de TI", nome)
    return nome

In [47]:
df['Carreiras'] = df['Carreiras'].apply(transformar)

In [51]:
def trasnforme(nome2): #por algum motivo, não consegui tratar a unica linha abaixo, por isso criei outro
    nome2 = re.sub(r"\bAssist Técnico De Gestaoem Sa\b", 'Assistente Técnico De Gestão Em Saúde', nome2)
    return nome2

In [52]:
df['Carreiras'] =  df['Carreiras'].apply(trasnforme)

In [53]:
df['Carreiras'].unique() 

array(['Assistente Em Administração',
       'Professor Ensino Básico Técnico Téc.',
       'Técnico De Laboratório-Área', 'Policial Rodoviário Federal',
       'Auxiliar Em Administração', 'Técnico De Tecnologia Da Informação',
       'Agente De Polícia Federal 3A C',
       'Técnico em Informações Geográficas E Estatíticas',
       'Administrador', 'Técnico Em Assuntos Educacionais',
       'Técnico Do Seguro Social', 'Médico-Área', 'Tecnologista',
       'Técnico Em Contabilidade', 'Engenheiro-Área',
       'Assistente De Aluno', 'Bibliotecário Documentalista',
       'Analista De Tecnologia Da Informação',
       'Auditor Fiscal Federal Agropecuário',
       'Tradutor Intérprete De Língua', 'Contador',
       'Técnico Em Enfermagem', 'Enfermeiro-Área', 'Tecnólogo-Formação',
       'Pedagogo-Área', 'Auxiliar De Enfermagem',
       'Indigenista Especializado', 'Procurador Da Fazenda',
       'Agente Administrativo', 'Psicólogo-Área', 'Advogado Da União',
       'Assistente Social', '

In [55]:
#Padronizando com nomes de colunas existentes no GBQ
df = df.rename(columns={'Carreiras': 'carreiras_area', "Ampla Concorrência": "ampla_concorrencia", "Cotista" : "forma_identificacao", "Total" : "quantidade_pessoas", "% Cotista" : "per_cotistas"})
df

,carreiras_area,ampla_concorrencia,forma_identificacao,quantidade_pessoas,per_cotistas
0,Assistente Em Administração,5019,1271,6290,"0,20"
1,Professor Ensino Básico Técnico Téc.,4357,571,4928,"0,12"
2,Técnico De Laboratório-Área,1411,163,1574,"0,10"
3,Policial Rodoviário Federal,763,176,939,"0,19"
4,Auxiliar Em Administração,707,174,881,"0,20"
...,...,...,...,...,...
223,Técnico Em Propriedade Industrial,1,O,1,"0,00"
224,Técnico Em Reabilitação Fisioterapêutica,1,O,1,"0,00"
225,Técnico Especializado II,1,O,1,"0,00"
226,Técnico de Suporte Infraestrutura de TI,1,O,1,"0,00"


In [ ]:
df['per_cotistas'] = df['per_cotistas'].str.replace(',','.').astype(float) #transformando em float

In [60]:
df['per_cotistas'] = df['per_cotistas']/100

In [62]:
df['forma_identificacao'].unique() #tratanto a coluna 'forma_identicacao' 

array(['1271', '571', '163', '176', '174', '96', '100', '71', '88', '98',
       '157', '10', '70', '39', '25', '54', '59', '42', '60', '34', '49',
       '55', '48', '26', '28', '58', '45', '9', '21', '22', '20', '16',
       '27', '6', '30', '15', '13', '18', '8', '12', '31', '5', '7', '4',
       '11', '3', '2', '1', 'O', ' O\n0,00', '1\n1,00', 'O\n0,00'],
      dtype=object)

In [69]:
df['forma_identificacao'] = df['forma_identificacao'].replace(['O', ' O\n0,00', 'O\n0,00'], '0')
df['forma_identificacao'] = df['forma_identificacao'].replace('1\n1,00', '1')

In [70]:
df

,carreiras_area,ampla_concorrencia,forma_identificacao,quantidade_pessoas,per_cotistas
0,Assistente Em Administração,5019,1271,6290,0.0020
1,Professor Ensino Básico Técnico Téc.,4357,571,4928,0.0012
2,Técnico De Laboratório-Área,1411,163,1574,0.0010
3,Policial Rodoviário Federal,763,176,939,0.0019
4,Auxiliar Em Administração,707,174,881,0.0020
...,...,...,...,...,...
223,Técnico Em Propriedade Industrial,1,0,1,0.0000
224,Técnico Em Reabilitação Fisioterapêutica,1,0,1,0.0000
225,Técnico Especializado II,1,0,1,0.0000
226,Técnico de Suporte Infraestrutura de TI,1,0,1,0.0000


In [75]:
df['ampla_concorrencia'].unique() #a letra 'O' também está presente aqui

array(['5019', '4357', '1411', '763', '707', '702', '588', '534', '493',
       '385', '226', '372', '293', '307', '315', '280', '264', '272',
       '236', '258', '203', '197', '213', '198', '159', '167', '165',
       '179', '157', '155', '138', '136', '89', '106', '124', '98', '113',
       '114', '107', '110', '108', '97', '83', '73', '68', '70', '69',
       '45', '65', '66', '64', '63', '51', '62', '53', '56', '54', '50',
       '47', '41', '44', '38', '39', '46', '36', '35', '40', '31', '33',
       '32', '28', '27', '25', '26', '30', '29', '21', '24', '18', '19',
       '22', '20', '14', '15', '17', '16', '11', '12', '13', '9', '8',
       '7', '6', '5', '1', '3', '4', 'O', '2'], dtype=object)

In [76]:
df['ampla_concorrencia'] = df['ampla_concorrencia'].replace('O', '0')

In [77]:
df['ampla_concorrencia'].unique() 

array(['5019', '4357', '1411', '763', '707', '702', '588', '534', '493',
       '385', '226', '372', '293', '307', '315', '280', '264', '272',
       '236', '258', '203', '197', '213', '198', '159', '167', '165',
       '179', '157', '155', '138', '136', '89', '106', '124', '98', '113',
       '114', '107', '110', '108', '97', '83', '73', '68', '70', '69',
       '45', '65', '66', '64', '63', '51', '62', '53', '56', '54', '50',
       '47', '41', '44', '38', '39', '46', '36', '35', '40', '31', '33',
       '32', '28', '27', '25', '26', '30', '29', '21', '24', '18', '19',
       '22', '20', '14', '15', '17', '16', '11', '12', '13', '9', '8',
       '7', '6', '5', '1', '3', '4', '0', '2'], dtype=object)

In [78]:
#transformando os valores de ambas colunas em integer
df['ampla_concorrencia'] = df['ampla_concorrencia'].astype(int) 
df['forma_identificacao'] = df['forma_identificacao'].astype(int)

In [79]:
df.dtypes

carreiras_area          object
ampla_concorrencia       int32
forma_identificacao      int32
quantidade_pessoas       int64
per_cotistas           float64
dtype: object

In [ ]:
##Fim do rascunho

# Subindo para o GBQ

In [135]:
df

,carreiras_selecionadas,tipo_ingresso,Total,Carreiras,valor,prop
0,ADMINISTRADOR,Ampla Concorrência,581.0,ADMINISTRADOR,493.0,0.848537
1,ADMINISTRADOR,Cotista,581.0,ADMINISTRADOR,88.0,0.151463
2,AGENTE DE POLICIA FEDERAL 3A C,Ampla Concorrência,688.0,AGENTE DE POLICIA FEDERAL 3A C,588.0,0.854651
3,AGENTE DE POLICIA FEDERAL 3A C,Cotista,688.0,AGENTE DE POLICIA FEDERAL 3A C,100.0,0.145349
4,ASSISTENTE EM ADMINISTRACAO,Ampla Concorrência,6290.0,ASSISTENTE EM ADMINISTRACAO,5019.0,0.797933
...,...,...,...,...,...,...
451,TECNICO DE LABORATORIO AREA,Cotista,1574.0,TECNICO DE LABORATORIO AREA,163.0,0.103558
452,TECNICO DO SEGURO SOCIAL,Ampla Concorrência,383.0,TECNICO DO SEGURO SOCIAL,226.0,0.590078
453,TECNICO DO SEGURO SOCIAL,Cotista,383.0,TECNICO DO SEGURO SOCIAL,157.0,0.409922
454,TECNICO EM ASSUNTOS EDUCACIONA,Ampla Concorrência,483.0,TECNICO EM ASSUNTOS EDUCACIONA,385.0,0.797101


In [187]:
client = bigquery.Client()
dataset_ref = client.dataset('acoes_afirmativas')

In [190]:
df['carreiras_selecionadas'].unique()

array(['Administrador', 'Agente De Policia Federal 3A C',
       'Assistente Em Administracao', 'Aux Em Administracao', 'Outros',
       'Policial Rodoviario Federal', 'Professor Ens Basico Tecn Tecn',
       'Tec De Tecnologia Da Informaca', 'Tec Infor Geograficas E Estati',
       'Tecnico De Laboratorio Area', 'Tecnico Do Seguro Social',
       'Tecnico Em Assuntos Educaciona'], dtype=object)

In [193]:
schema = [bigquery.SchemaField('carreiras_selecionadas', 'STRING', description= 'Grupo de carreiras selecioandas para o gráfico'),
          bigquery.SchemaField('tipo_ingresso', 'STRING', description= 'tipo de ingresso'),
          bigquery.SchemaField('Total', 'FLOAT', description= 'Total daquela carreira. Para o outros é um agregado.'),
          bigquery.SchemaField('Carreiras', 'STRING', description= 'Carreira original'),
          bigquery.SchemaField('valor', 'FLOAT', description= 'valor de ingressantes'),                    
          bigquery.SchemaField('prop', 'FLOAT', description= 'Proporção por tipo de ingresso em relação ao total de ingressos naquela carreira. Para outros ela é apenas um agregado')]

In [194]:
table_ref = dataset_ref.table('ENAP_corte_carreiras')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=422353db-b5c7-4eb0-a321-8628d84484a9>